In [ ]:
import pandas as  pd 
import numpy as np 
import seaborn as sns 
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from imblearn.pipeline import make_pipeline
from collections import Counter
from sklearn.metrics import RocCurveDisplay, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline
sns.set_style("whitegrid")

Welcome to this new post, these are the most important thing in this notebook
1. Feature Engineering techniques to fill nan Values
2. Unsupervise Machine Learning for Outlier detection
3. Pipeline + Cross_validation
4. Technique to handle Imbalanced Data
5. Avance Pipeline + AUR Curve


Thanks for passing by... I am really gld to share this with the community , ***please leave a comment or vote up 
I am one medal away to be notebook Expert and I am working on my CV / Potofolio etc ***

# Obtain Data

In [ ]:
path = '../input/hr-analytics-job-change-of-data-scientists/aug_train.csv'
# Loading data
df = pd.read_csv(path)
df.describe(include= 'object')

# Scrub Data

Based on my studies in MBA a start-up usually have lot of challeging 'money wise' 
Usually they look for MAster or phd for intership 
therefore i asume that if you are MAster/phd with more than 10 years your salsary will be a good one 
ine that a start up wont pay becaue the financial difficulties and if you stay ther emore thant 3 years means that it is not a start up anymore 

In [ ]:
### Fuctions

def inputing_data_major (cols):
    level,enrolled,major,exp = cols[0],cols[1],cols[2],cols[3]
    
    if pd.isnull(major):
        if all([level in ['Primary School','High School'], exp  > 5]):
            return 'No Major'
        elif all([level in['Graduate','Masters','High School','Phd'], exp > 1]):
            return 'Other'
        else:
            return major
    return major

def inputing_data (cols):
    level,enrolled,major,exp = cols[0],cols[1],cols[2],cols[3]
    
    if pd.isnull(enrolled):
        if all([level in ['Primary School','High School'], exp  > 0]):
            return 'no_enrollment'
            #When other can be he /she could go to university
        else: 
            return enrolled 
    else:
        return enrolled
    
    
# Based on my studies in MBA a start-up usually have lot of challeging 'money wise' 
# Usually they look for MAster or phd for intership 
#therefore i asume that if you are MAster/phd with more than 10 years your salsary will be a good one 
#ine that a start up wont pay becaue the financial difficulties and if you stay ther emore thant 3 years means that it is not a start up anymore 


def inputing_data_company_type (cols):
    comp,level,last_job,exp = cols[0],cols[1],cols[2],cols[3]
    
    if pd.isnull(comp):
        if all([level in ['Mastersl','Phd','Graduate'], last_job in ['3','4','>4'],exp  > 10]):
            return 'Pvt Ltd'
            #When other can be he /she could go to university
        else: 
            return comp 
    else:
        return comp
    
    
def company_size (cols):
    comp_size,comp = cols[0],cols[1]
    
    comps = {'Early Stage Startup':'<10','Funded Startup':'50-99','NGO':'100-500',
            'Other':'100-500','Public Sector':'1000-4999','Pvt Ltd':'50-99'}
    
    if pd.isnull(comp_size):
        if comp in comps.keys():
            return  comps[comp]
            #When other can be he /she could go to university
        else: 
            return comp_size 
    else:
        return comp_size 
    
    
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)
    

In [ ]:
# I will use this information to create bin / segmentation to Fill nan Values
df.company_size.groupby(df['company_type']).apply(lambda g: g.mode()).reset_index()[['company_type', 'company_size']].values
import missingno as msno
msno.matrix(df, sparkline=True, figsize=(10,5), sort='ascending', fontsize=12, labels=True, color=(0.25, 0.45, 0.6));
print("Missing values")

###### Experienced 
I will use this columns as condition to fill some nan values that i consider can be fill out

In [ ]:
experiences_values = {'<1':0.99,'>20':21}
df.experience.replace(experiences_values, inplace=True)
df.experience = df.experience.astype(float)

###### Dealing with gender
I assumed that 'nan values' in gender are those candidate that didnt want to provide that info, there fore fo to Others

In [ ]:
df['gender'].fillna('Other', inplace = True)

###### Company size
Those companies between 10-99 are start up
Those row with univeristy_enrollment  = nan , experience = nan , last_new_job = never are equal to a new categories NW = 'Never Work' 
we dont have that info not because was a mistake but because they never work

In [ ]:
comps = {'Early Stage Startup':'<10','Funded Startup':'50-99','NGO':'100-500',
            'Other':'100-500','Public Sector':'1000-4999','Pvt Ltd':'50-99','middle_size_st':'10/49'}

for key,values in comps.items():
    df.loc[((df['company_type'].isna())&(df['company_size'] ==values)),'company_type'] = key
    
df.loc[( (df['company_type'].isna()) & (df['company_size'].isna()) & (df['last_new_job']=='never') ),['company_type','company_size'] ]= 'NW'

###### Company size
Other columns - Please check Fuctions Section

In [ ]:
df.major_discipline = df[['education_level','enrolled_university','major_discipline','experience']].apply(inputing_data_major,axis=1)
df.enrolled_university = df[['education_level','enrolled_university','major_discipline','experience']].apply(inputing_data,axis=1)
df.company_type = df[['company_type','education_level','last_new_job','experience']].apply(inputing_data_company_type,axis=1)
df.company_size = df[['company_size','company_type']].apply(company_size,axis=1)
msno.matrix(df, sparkline=True, figsize=(10,5), sort='ascending', fontsize=12, labels=True, color=(0.25, 0.45, 0.6));
print("After feature Engineering")

###### others
This are the row that have more than 4 nan values so it is Imposible to make any connection between columns

In [ ]:
qty_of_nuls =1
df.drop(df.iloc[df[(df.isnull().sum(axis=1) >= qty_of_nuls)].index].index, inplace=True)
msno.matrix(df, sparkline=True, figsize=(10,5), sort='ascending', fontsize=12, labels=True, color=(0.25, 0.45, 0.6))

---------------No more Nan, Null Values ---------------------

# Explore

In [ ]:
# Frist Visualizatoin 
vs = df.copy()

fig,ax = plt.subplots(1,3,figsize = (18,5))

#training time vs target 
sns.boxenplot(y='training_hours' , x='gender', data= vs , hue= 'target', ax=ax[0] )
sns.countplot(y='gender'  ,data= vs, hue='target', ax=ax[1])
sns.kdeplot(data = vs , x='training_hours', hue = 'gender',multiple="stack", ax=ax[2])
print("The average Training required to be change/stay in a job seems to be the same by gender")
print("Male are more intersted in staying in the same job")
print('the difference between training time is obvious, male are the one with more training time - experience')

# Model

In [ ]:
from IPython.display import display
from PIL import Image
path=('../input/diagram/diagram.png')
display(Image.open(path))

print("\nIn here a small diagram to better understand how I filled those nan Values")

### Dropping columns

I strongly believe that city and city development  are related , therefore I can choose the numerical value (index) to avoid high dimensionality

In [ ]:
df.drop(['enrollee_id','city'], axis = 1, inplace=True)
df.reset_index(drop=True,inplace=True)

### Segmented data set into Categorical and Numerical 

In [ ]:
target = df['target']
numerical = df[['city_development_index','training_hours']] 
ordinal = df[['education_level','experience','company_size','last_new_job']] 
enco= df[['gender','relevent_experience','major_discipline','enrolled_university']]

####  Analysis, Consideration and Transformation.

1. Identify and Transform data into ML standard input - No string, No nan, No-Null
2. Identify Outliers - Use unsupervised ML OUTLIERS
3. Transform numerical Variables if they are skewed
3. Define the best algorithms and Strategies
4. Define the best strategy to deal with imbalanced data set
5. Hyperparameter

***Categorical Ordinal*** 
* ***education_level***,***experience***,***company_size***,***last_new_job***
- --For Experiences-- 
I asuumed that the more experienced you are the higher the rank you get - but sure I also Understand that is not always like that - 
I create 8 bin that goes from [[0, 3) < [3, 6) < [6, 9) < [9, 12) < [12, 15) < [15, 18) < [18, 21) < [21, 22)] and 0 - 8 respectively

- --For Education Level-- 
The Higher Education the Higher Rank 

- --For Company Size-- 
This is a bit controversial because there is not a studies that support my theory but based on my studies in MBA and after consulting with   @Mauricio - https://www.linkedin.com/in/mauricio-rodr%C3%ADguez-082170109/-   expert in Human Resources and Recruitment 
We concluded that the average attrition rate known as  "turnover" in Start-up  is higher than in well-established company which suggest that the chances of quitting and go to other companies is higher, and because this study is about that i assigned values from  -1-8 where 8  is the best one which means your chances of leave the company are lower

***Categorical Nominal*** 
* ***city_development_index***,***training_hours*** 
find their skewness and apply tranformation if it is needed

***Enconding*** 
* ***gender***,***relevent_experience***,***major_discipline***,***enrolled_university*** 
I will use OneHotEncoder to transformed this variables into Categorical to avoid any preferences 

***Final notes*** : I could use LabelEnconder from scikit-learn but it doesnt have a way to define the order of importances -- actually it has but was easier do it manually -

##### 1. Identify and Transform data into ML standard input - No string, No nan, No-Null

###### Ordinal

In [ ]:
# For Experiences 
bins,labels = [0,3,6,9,12,15,18,21,22],[0,1,2,3,4,5,6,7]
ordinal.loc[:,'experience'] = pd.cut(ordinal.loc[:,'experience'], bins = bins, labels=labels, right = True).values.astype(int)

# For Education Level
levels_ = {'Primary School':0,'High School':1,'Graduate':2 ,'Masters':3,'Phd':4}
ordinal.loc[:,'education_level'].replace(levels_, inplace= True)

# For Company Size - 
c_size = {'50-99':3, 'NW':0, '<10':1, '10000+':8, '5000-9999':7, '1000-4999':6, '10/49':2,'100-500':4, '500-999':5}
ordinal.loc[:,'company_size'].replace(c_size, inplace= True)

#For Last New Job
last_job = {'>4':3, 'never':0, '4':2, '1':1, '3':2, '2':1}
ordinal.loc[:,'last_new_job'].replace(last_job, inplace= True)

display_side_by_side(ordinal.head(5), df[['education_level','experience','company_size','last_new_job']].head(5))

###### Nominal

In [ ]:
from sklearn.preprocessing import OneHotEncoder
df_enc = enco.copy()
OHC = OneHotEncoder()

final = pd.DataFrame()
for col in enco:
    new_data = OHC.fit_transform(df_enc[[col]])
    df_enc = df_enc.drop(col, axis =1)
    cats = OHC.categories_
    new_cols = ["_".join([col,cat]) for cat in cats[0]]
    new_df = pd.DataFrame(new_data.toarray(), columns = new_cols) 
    final = pd.concat([final,new_df], axis=1)

final

##### 2. Identify Outliers

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

#Joining Data set
data = pd.concat([numerical, ordinal,final,target], axis=1)

#Identifying and Ensembling to Outliers-data set
LOF = LocalOutlierFactor(novelty=False)
data['Outliers'] = LOF.fit_predict(data)

#Finding Ourliers
outliers = data[data['Outliers'] ==-1].index
print(f'Number of outliers {len(outliers)}')

#Visualizations
fig, ax = plt.subplots(1,3 ,figsize=(15,5))
out_liers = data.iloc[outliers]

sns.boxenplot(x='target' ,y ='experience' , data=data, ax=ax[0])
sns.scatterplot(x='target' ,y ='experience' , data=out_liers, color ='r',ax =ax[0])

sns.boxenplot(x='target' ,y ='city_development_index' , data=data, ax=ax[1])
sns.scatterplot(x='target' ,y ='city_development_index' , data=out_liers, color ='r', ax=ax[1])

sns.boxenplot(x='target' ,y ='training_hours' , data=data, ax=ax[2])
sns.scatterplot(x='target' ,y ='training_hours' , data=out_liers, color ='r', ax=ax[2])

print("Let's Visualize the outliers in Training hours, Experience and City")
print("Notice that the 'red' dotes are the outliers, consider LocalOutlierFactor is finding those outliers \
in high dimmensional space, and we are visualizing them in 2D")
data.drop('Outliers', axis = 1 , inplace = True)
data.drop(outliers , inplace=True)

##### 3.Transform numerical Variables if they are skewed

In [ ]:
f,ax = plt.subplots(2,4, figsize = (17,10))
sns.color_palette("rocket", as_cmap=True)

#------Numerical values 
numerical = data[['city_development_index','training_hours']]
x='city_development_index'
x1= 'training_hours'
title ='Training hours Distribution Min, Mean, Max'

# Before transforming 
sns.histplot(x=x, data= numerical, ax = ax[0,0])
sns.boxplot(y=x, data =numerical , orient ='v' , ax=ax[0,1],palette="Paired").set(title=title)
sns.histplot(x=x1, data= numerical, ax = ax[1,0])
sns.boxplot(y=x1, data =numerical , orient ='v' , ax=ax[1,1],palette="flare").set(title=title)

# After Transforming 
for i in numerical.columns:
    data[i] = np.log1p(data[i]).astype('float')
    
sns.histplot(x=x, data= data, ax = ax[0,2], element="step",color ='orange')
sns.boxenplot(y=x, data =data , orient ='v' , ax=ax[0,3], palette="viridis").set(title=title)
sns.histplot(x=x1, data= data, ax = ax[1,2],element="poly", color ='orange')
sns.boxenplot(y=x1, data =data , orient ='v' , ax=ax[1,3],palette="rocket").set(title=title);

##### 4. Define the best algorithms and Strategies 
* I chose the following algorithms because the nature of the data , classification task, training time , the ability of classifying data in medium - high dimensionality space

In [ ]:
#Data Set
data.reset_index(drop=True, inplace=True)

#Algorithms Default Parameters
KNN = KNeighborsClassifier(n_neighbors=3)
RF = RandomForestClassifier(max_depth = 30) 
GB = GradientBoostingClassifier(n_estimators=100, random_state=42)
SV = SVC()
LSV = LinearSVC() #doesnt give the Probability 
LR = LogisticRegression()

In [ ]:
# Class Relation

sns.relplot(y='training_hours', x = 'city_development_index', hue='target',size='education_level',
            data =data ,palette='viridis', sizes=(10, 200),alpha=.6,height=6);

print("Before choosing the best way to handle the data set , we have to get familiar with its distributions \
in here we can notice that out target -the chances looking/ no looking for a job - will not be easy to classify")

print("So the first question would be : which model will help us to do this job \
do I want to Downsample, oversample , use a technique that combines both or just go with the skelarn approach, there is not a durect answer but we had\
some clues : 1. the target are mixed , if we use SMOTE to create Synthetic samples and this kind of data set is perfect for that algorithm, 2. We can use Cross validation and AUC to choose the best strategy")

#####  5. Define the best strategy to deal with imbalanced data set

###### Oversampling

this consist on create/copy / repeat samples from the category that has less observation in this case "1" I will use ADASYN, RandomOverSampler, SMOTE + Cross validation 
There is a good example in Imblanaced-Sklearn

1. Visualize
2. Stratified
3. Choose the model
4. Plot curve

In [ ]:
from imblearn.over_sampling import ADASYN, RandomOverSampler, SMOTE

# Splitting
X = data.drop(['target'], axis = 1)
y = data[['target']]
cv = StratifiedKFold(n_splits=3)
pos_label = y.target.value_counts(normalize=True).idxmin()

In [ ]:
# Creating List to Update Values
disp_default = [] # <- default Values
disp_var = [] #<- OverSampling 
score = []  #<-Scores

#Creating And Initiating
pipeline = [
    # NO oversampling Techniques
    make_pipeline(GB),
    make_pipeline(RF),
    make_pipeline(KNN),
    make_pipeline(StandardScaler(),LR), 
    
    ## oversampling Techniques
    make_pipeline(SMOTE(random_state=42,sampling_strategy ='minority'), GB),
    make_pipeline(ADASYN(random_state=42), RF),
    make_pipeline(SMOTE(random_state=42), RF),
    make_pipeline(ADASYN(random_state=42), KNN),
    make_pipeline(SMOTE(random_state=42), KNN),
    make_pipeline(ADASYN(random_state=42),StandardScaler(), LR),
    make_pipeline(SMOTE(random_state=42),StandardScaler(), LR)]

# Starting 
for model in pipeline:
    #Mean ROC curve
    mean_tpr, mean_fpr = 0.0, np.linspace(0, 1, 100)
    for train, test in cv.split(X, y):
        
        #Training Models - < Probability and Final Outputs
        model.fit(X.iloc[train], y.iloc[train].values.ravel())
        
        y_proba = model.predict_proba(X.iloc[test])
        y_pred = model.predict(X.iloc[test])

        
        #Creating Dimmensions
        pos_label_idx = np.flatnonzero(model.classes_ == pos_label)[0]
        fpr, tpr, thresholds = roc_curve(y.iloc[test], y_proba[:, pos_label_idx], pos_label=pos_label)
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0

    mean_tpr /= cv.get_n_splits(X, y)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    
    #------------IF AUC > 63 Pass
    if (model[0].__class__.__name__ =='SMOTE') or (model[0].__class__.__name__ =='ADASYN'):
        if mean_auc > .63:
            disp_var.append(RocCurveDisplay(
                    fpr=mean_fpr,tpr=mean_tpr,
                    roc_auc=mean_auc,estimator_name=f"{model[0].__class__.__name__}"+"_"+f"{model.steps[-1][0]}"))
            try:
                model['smote'].fit_resample(X,y)
                print(f"{model[0].__class__.__name__}"+"_"+f"{model.steps[-1][0]}")
                print('Previous dataset shape %s' % Counter(data['target']))
                print('New Data dataset shape %s' % Counter(y_res['target']), "<-------Here new data")
                print("\n")
            except: 
                pass
    else:
        disp_default.append(RocCurveDisplay(
                fpr=mean_fpr,tpr=mean_tpr,
                roc_auc=mean_auc, estimator_name=f"{model[0].__class__.__name__}"))
        
    # ----------Scores
    y_true = y.iloc[test]
    Pre_ = precision_score(y_true,y_pred,average='macro')
    Rec_ = recall_score(y_true,y_pred,average='macro')
    F1 = f1_score(y_true, y_pred,average='macro')   
    name =f"{model.steps[0][0]}"+"_"+f"{model.steps[-1][0]}"
    score.append(pd.Series({'name':name ,'Pre':Pre_,'Rec_':Rec_,'F1':F1}))
    

In [ ]:
# ---- Plotting
fig, ax = plt.subplots(1,3,figsize=(20, 10))
fig.suptitle("Comparison of over-sampling methods with a KNN, RandomForest, Logistical Regression")
ax[0].set_xlim([0, 1]) ,ax[1].set_xlim([0, 1]) 


#Visualization of classes
sns.countplot(x='target', data=data, ax=ax[0])
ax[0].plot([0, 1], [0, 1])

for i in disp_default:
    sns.despine(offset=10, ax=ax[0])
    i.plot(ax=ax[1], linestyle="--")
    ax[1].plot([0, 1], [0, 1], linestyle="--", color="k")
    
for j in disp_var:
    sns.despine(offset=10, ax=ax[0])
    j.plot(ax=ax[2], linestyle="--")
    ax[2].plot([0, 1], [0, 1], linestyle="--", color="k")
    

    
print("As we can observed KNN was eliminated because the AUC wasnt greater than 65 , another observation is that gradient boosting has better performance \
wihout Oversampling but we MUST analize Recall, Precision and that is exactly what you are going to see in the next section\
Remember each model is unique and sometime accuracy is no the best measurement")


In [ ]:
final = pd.concat(score, axis=1).T
final.sort_values(['Pre','Rec_','F1'], ascending=False)

# Conclusion

This is the most crucial part , as I said before accuracy is not always the best way to measure  a model 
in this example we have imbalanced model bias to 1 "Looking for a job change" so the model itself would be great at
identifying people who is looking for a job change, as the description said
***wants to hire data scientists among people who successfully pass some courses which conduct by the company***, there fore precision is the most relevant measurement , because it can predict correctly 71% of the time , but how about if it were in the other way around let say the company XXX end people to this courses and they want to make sure they wont leave the company for this new one , in this case the most important measurement in rec_ that can be understood as "the ability of the model to identify those who doesn't want to change their job"

But this alway up to the Team who is taking care of this issue.
and how about you..!! what would you do? and how would you explain this to your manager?


*** Another Observations ***
- Downsampling technique can be implemented as well - "I will do in few weeks"
- we can also find better parameters for gradientboostingclassifier such as Learning rate
- In real life would be better to collect more data and make sure about people response.

Hope you can learn something from this notebook ...!!!
any feedback / Comment is more than welcome 